In [16]:
import pandas as pd
import spacy

**Uploaded coutrnies df to Pt2 to start the NER relationship process**

In [18]:
countries_df = pd.read_csv("C:\\Users\\Kathe\\Documents\\Project 1 - Visualizations for Data Analysts\\1.-Data-Visualization-in-Python\\countries_list_20th_century_1.5.csv")

In [20]:
print(countries_df)

     Unnamed: 0                          country_name
0             1                          Afghanistan 
1             2                              Albania 
2             3                              Algeria 
3             4                              Andorra 
4             5                               Angola 
..          ...                                   ...
204         205     Sahrawi Arab Democratic Republic 
205         206                           Somaliland 
206         207                        South Ossetia 
207         208                               Taiwan 
208         209                          Transnistria

[209 rows x 2 columns]


**Start Cleaning proccess to eliminate unwanted columns**

In [23]:
countries_list = countries_df['country_name'].tolist()

In [52]:
def filter_entity(ent_list, character_df):
    return [ent for ent in ent_list if ent in countries_list]

**Normalize the country names (strip extra spaces, lowercase) for matching**

In [54]:
countries_list = [country.strip().lower() for country in countries_list]

**Uploaded tokens_text saved in "1.6 wrangled_txt-PT1"**

In [56]:
with open(r"C:\Users\Kathe\Documents\Project 1 - Visualizations for Data Analysts\1.-Data-Visualization-in-Python\1.6 Task Files\tokens_text.txt", "r") as file:
    tokenized_text = file.read()

In [62]:
print(type(tokenized_text))

<class 'str'>


In [60]:
nlp = spacy.load("en_core_web_sm")

**Use NLP from spacy to tokenize the text and label the text in the tokens_text file**

In [112]:
doc = nlp(tokenized_text)

In [113]:
df_sentences = []

In [114]:
# Loop through sentences, get entity list for each sentence
for sent in doc.sents:
       entity_list = [ent.text for ent in sent.ents]
       df_sentences.append({"sentence": sent, "entities": entity_list})

df_sentences = pd.DataFrame(df_sentences)

In [115]:
df_sentences.head()

,sentence,entities
0,"(headers, contents, key, events, th, century, ...","[world war ii, thursday, december, year, today..."
1,"(caused, war, see, stabintheback, legend, germ...","[germany, europe, yugoslavia, austrohungarian ..."
2,"(acting, guarantees, territory, secret, part, ...","[germany, republics, estonia, latvia, lithuania]"
3,"(well, eastern, poland, would, remain, soviet,...","[poland, soviet, finland, soviet]"
4,"(staunch, finnish, resistance, became, known, ...","[finnish, winter, germany, soviet union, defea..."


In [116]:
df_sentences['countries_list'] = df_sentences['entities'].apply(lambda x: filter_entity(x, None))

In [117]:
# Filter out sentences that don’t have any character entities

df_sentences_filtered = df_sentences[df_sentences['countries_list'].map(len) > 0]

df_sentences_filtered.tail(10)

,sentence,entities,countries_list
1,"(caused, war, see, stabintheback, legend, germ...","[germany, europe, yugoslavia, austrohungarian ...","[germany, italy, germany, germany, united stat..."
2,"(acting, guarantees, territory, secret, part, ...","[germany, republics, estonia, latvia, lithuania]","[germany, estonia, latvia, lithuania]"
3,"(well, eastern, poland, would, remain, soviet,...","[poland, soviet, finland, soviet]","[poland, finland]"
4,"(staunch, finnish, resistance, became, known, ...","[finnish, winter, germany, soviet union, defea...","[germany, netherlands, norway, france, france,..."
5,"(defeat, montgomery, el, alamein, first, decis...","[first, germany army, soviet, united states, b...","[united states, morocco, algeria]"
6,"(july, allies, invaded, sicily, preparation, a...","[july, italy, winston churchill called, septem...","[italy, italy, italy, italy, france, germany, ..."
7,"(australian, city, darwin, first, attack, fore...","[australian, darwin, first, australian, first,...","[united states, japan, japan, japan, india, ja..."
8,"(several, differing, views, exist, regarding, ...","[jews, night night, germans, third, jews, jews...","[poland, united states, mexico, germany, unite..."
9,"(yuri, gagarin, us, pulled, ahead, eventually,...","[yuri, apollo, apollo, july, five, apollo, sov...","[united states, russia, japan, canada, russia]"
12,"(large, systems, universities, efforts, became...","[two, al gore, america, tim bernerslee, centur...",[india]


In [118]:
# Take only the first name of the characters

df_sentences_filtered['countries_list'] = df_sentences_filtered['countries_list'].apply(lambda x: [item.split()[0]
                                                                                                    for item in x])

C:\Users\Kathe\AppData\Local\Temp\ipykernel_1120\140596092.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentences_filtered['countries_list'] = df_sentences_filtered['countries_list'].apply(lambda x: [item.split()[0]


In [119]:
# Defining relationships

# window size = 5 : this defines how many sentences will be looked at simultaneously
relationships = [] # create an empty list

for i in range(df_sentences_filtered.index[-1]):
    end_i = min(i+5, df_sentences_filtered.index[-1])
    char_list = sum((df_sentences_filtered.loc[i: end_i].countries_list), [])

     # Remove duplicated characters that are next to each other
    char_unique = [char_list[i] for i in range(len(char_list))]
    if len(char_unique) > 1:
        for idx, a in enumerate( char_unique[ :-1]):
            b = char_unique[idx + 1]
            relationships.append({"source": a, "target": b})    

In [120]:
relationship_df = pd.DataFrame(relationships)

In [121]:
import numpy as np

In [122]:
# Sort the cases with a- >b and b- >a
relationships_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationships_df.head(5)

,source,target
0,france,russia
1,germany,russia
2,germany,russia
3,russia,russia
4,germany,russia


**Filter GPE entities and match them with the cleaned countries list**

**Create the list of each unique relationship by grouping them by each relationship between the two dfs and how many times they occur on the Wikipedia page**

In [125]:
relationships_df["value"] = 1
relationships_df = relationships_df.groupby(["source","target"], sort=False, as_index=False).sum().sort_values(by="value", ascending=False)

relationships_df.head(10)

,source,target,value
39,japan,japan,90
30,japan,united,28
49,united,united,23
3,germany,germany,22
31,germany,japan,22
5,germany,united,22
21,france,france,20
10,france,germany,20
71,vietnam,vietnam,18
44,india,japan,18


**Print the relationship chart between the two**

In [134]:
relationships_df=relationships_df.loc[relationships_df["source"] != relationships_df["target"]]

In [136]:
relationships_df

,source,target,value
30,japan,united,28
31,germany,japan,22
5,germany,united,22
10,france,germany,20
44,india,japan,18
...,...,...,...
13,estonia,latvia,3
12,estonia,germany,3
7,poland,spain,2
6,germany,spain,2


**Save to CSV file to folder**

In [138]:
file_path = r"C:\Users\Kathe\Documents\Project 1 - Visualizations for Data Analysts\relationships_count.csv"
relationships_df.to_csv(file_path, index=False)